In [40]:
import numpy as np
import logging
import gzip
import matplotlib.pyplot as plt

PWD="/home/duzicman/projects/tau/statistical_machine_learning/logistic_classifier/datasets"
DATASET = {"labels": {"path" :PWD + "/train-labels-idx1-ubyte.gz", "offset":8}, "data": {"path": PWD + "/train-images-idx3-ubyte.gz", "offset":16}}
IMAGE_LENGTH = 28
IMAGE_SIZE = IMAGE_LENGTH ** 2


In [41]:
def load_dataset(file_path, offset):
    with gzip.open(file_path, 'rb') as data_file:
        data = np.frombuffer(data_file.read(), np.uint8, offset=offset)
    return data

x_sample = load_dataset(DATASET["data"]["path"], DATASET["data"]["offset"])
t_labels = load_dataset(DATASET["labels"]["path"], DATASET["labels"]["offset"])


In [42]:
DATA_SIZE = 50
def one_hot_encoding(labels):
    label_ammount = np.max(labels) + 1
    return np.eye(label_ammount)[labels]

def format_dataset(x_sample, labels):
    labels = one_hot_encoding(labels)
    x_sample = np.reshape(x_sample, (-1, IMAGE_SIZE))
    x_sample = np.c_[x_sample, np.ones(x_sample.shape[0])]
    return x_sample, labels

x_sample, t_labels = format_dataset(x_sample, t_labels)
class_number = t_labels.shape[1]
#TODO remove partial data size
x_sample = x_sample[:DATA_SIZE] 
t_labels = t_labels[:DATA_SIZE]
 
x_sample = np.transpose(x_sample)

In [43]:
def show_image(image):
    pixels = np.reshape(image, (IMAGE_LENGTH, IMAGE_LENGTH))
    plt.imshow(pixels, cmap='gray')
    plt.show()
# show_image(features[0])
#TODO I added extra normilization pixel so it doesnt work noe
# print(labels[0])


In [53]:
def cross_entropy_loss(y_labels_probabilites, t_labels) -> float:
    natural_log = lambda x: np.log(x)
    y_logged = np.matrix.transpose(np.vectorize(natural_log)(y_labels_probabilites))
    return -np.trace(np.matmul(t_labels, y_logged))

def caculate_lables_probabilities(w_weights, x_sample):
    a_logits = np.matmul(np.transpose(x_sample), w_weights)
    exponent = lambda x: np.exp(x)
    exponent_logits = np.vectorize(exponent)(a_logits)
    row_summation = np.sum(exponent_logits, axis=1)[:, np.newaxis]
    y_labels_probabilities = exponent_logits /  row_summation
    return y_labels_probabilities
#TODO rename probablity to a single name
    
w_weights = np.zeros((IMAGE_SIZE + 1, class_number))
y_label_probabilites = caculate_lables_probabilities(w_weights, x_sample)
print(y_label_probabilites)

[[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0